In [9]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [3]:
# Replace these with your database details
dbname = "moodmap_db"
user = "postgres"
password = "9b6b09d4a1"
host = "localhost"
port = "5432"

# Create a connection string
conn_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

# Create a connection
engine = create_engine(conn_str)


In [ ]:
    # total_calls,                            |  +
    # satisfied_calls,                        | will add to db or calculate here
    # not_satisfied_calls,                    | will add to db or calculate here
    # alerts,                                 |  +
    # customer_sat_level,                     |  +  
    # agent_perf_rate,                        |  +
    # costumer_care,                          |  +

    # average_customer_voice_emotions,
    # average_agent_voice_emotions,
    # average_customer_text_emotions,
    # average_agent_text_emotions,
    # share_of_customer_voice_emotion_timeline,
    # share_of_agent_voice_emotion_timeline,
    # share_of_customer_text_emotion_timeline,
    # share_of_agent_text_emotion_timeline,

    # call_topics,                            |  +
    # ironic_count,                           |  +
    # not_ironic_count,                       |  +
    # hateful_count,                          |  +
    # targeted_count,                         |  - 
    # aggressive_count,                       |  -

    # Normal                                  |  +

In [3]:
def get_data_info(start_date,end_date,satisfaction_level,performance_level):
    # Replace this with your SQL query
    query_call = """SELECT * FROM public.call
                            WHERE starttime > '{s_d}' and endtime < '{e_d}'
                            and customer_satisfaction_rate >'{c_s_0}' and customer_satisfaction_rate <'{c_s_1}';""".format(s_d=start_date,e_d=end_date,c_s_0=satisfaction_level[0],c_s_1=satisfaction_level[1])

    # Replace this with your SQL query
    query_call_info = """SELECT * FROM public.callinfo"""

    df_call = pd.read_sql_query(query_call, engine)
    df_call_info = pd.read_sql_query(query_call_info, engine)

    df = pd.merge(df_call,df_call_info,how='left',on='callid')

    # Replace this with your SQL query
    query_clint_voice_emotion = """
                        SELECT * FROM public.call_mood_client
                        WHERE callid IN {}
                            """.format(tuple(df.callid))

    clint_voice_emotion = pd.read_sql_query(query_clint_voice_emotion, engine)

    # Replace this with your SQL query
    query_agent_voice_emotion = """
                        SELECT * FROM public.call_mood_agent
                        WHERE callid IN {}
                            """.format(tuple(df.callid))

    agent_voice_emotion = pd.read_sql_query(query_agent_voice_emotion, engine)

    # Replace this with your SQL query
    query_clint_text_emotion = """
                        SELECT * FROM public.call_sentence_client
                        WHERE callid IN {}
                            """.format(tuple(df.callid))

    clint_text_emotion = pd.read_sql_query(query_clint_text_emotion, engine)

    # Replace this with your SQL query
    query_agent_text_emotion = """
                        SELECT * FROM public.call_sentence_agent
                        WHERE callid IN {}
                            """.format(tuple(df.callid))

    agent_text_emotion = pd.read_sql_query(query_agent_text_emotion, engine)

       # Extract emotions and counts
    voice_emotions_clinet = pd.DataFrame(clint_voice_emotion.emotion.value_counts()).reset_index()['emotion'].tolist()
    voice_counts_client = pd.DataFrame(clint_voice_emotion.emotion.value_counts()).reset_index()['count'].tolist()

    # Extract emotions and counts
    voice_emotions_agent = pd.DataFrame(agent_voice_emotion.emotion.value_counts()).reset_index()['emotion'].tolist()
    voice_counts_agent = pd.DataFrame(agent_voice_emotion.emotion.value_counts()).reset_index()['count'].tolist()

    # Extract emotions and counts
    text_emotions_clinet = pd.DataFrame(clint_text_emotion.sentiment.value_counts()).reset_index()['sentiment'].tolist()
    text_counts_client = pd.DataFrame(clint_text_emotion.sentiment.value_counts()).reset_index()['count'].tolist()

    # Extract emotions and counts
    text_emotions_agent = pd.DataFrame(agent_text_emotion.sentiment.value_counts()).reset_index()['sentiment'].tolist()
    text_counts_agent = pd.DataFrame(agent_text_emotion.sentiment.value_counts()).reset_index()['count'].tolist()

    tatal_calls = len(df)
    alerts = df.alert.value_counts()[1]
    customer_sat_level = df.customer_satisfaction_rate.mean()
    agent_perf_rate = df.agent_performance_rate.mean()
    costumer_care = df.callscore.mean()
    call_topics = df[['topic']]
    ironic_count = np.where(df.ironypercent > 30,1,0).sum()
    not_ironic_count = (1 - np.where(df.ironypercent > 30,1,0)).sum()
    hateful_count = np.where(df.hatespeechpercent > 30,1,0).sum()
    not_hateful_count = (1 - np.where(df.hatespeechpercent > 30,1,0)).sum()

    return voice_emotions_clinet,voice_counts_client,voice_emotions_agent,voice_counts_agent,text_emotions_clinet,text_counts_client,text_emotions_agent,text_counts_agent,tatal_calls,alerts,customer_sat_level,agent_perf_rate,costumer_care,call_topics,ironic_count,not_ironic_count,hateful_count,not_hateful_count

In [4]:
# Replace this with your SQL query
query_call = """SELECT * FROM public.call;"""
# Replace this with your SQL query
query_call_info = """SELECT * FROM public.callinfo"""
df_call = pd.read_sql_query(query_call, engine)
df_call_info = pd.read_sql_query(query_call_info, engine)
df = pd.merge(df_call,df_call_info,how='left',on='callid')

In [5]:
list(df_call_info.columns)

['id',
 'callid',
 'topic',
 'ironypercent',
 'clientinterrupts',
 'agentinterrupts',
 'silencepercent',
 'hatespeechpercent',
 'clientspeechduration',
 'agentspeechduration',
 'customer_satisfaction_rate',
 'agent_performance_rate',
 'alert',
 'satisfied']

In [ ]:
['id',
 'callid',
 'topic',
 'ironypercent',
 'clientinterrupts',
 'agentinterrupts',
 'silencepercent',
 'hatespeechpercent',
 'clientspeechduration',
 'agentspeechduration',
 'customer_satisfaction_rate',
 'agent_performance_rate',
 'alert',
 'satisfied']

In [1]:
df

NameError: name 'df' is not defined

In [6]:
# Replace this with your SQL query
query_clint_voice_emotion = """
                    SELECT * FROM public.call_mood_client
                    WHERE callid IN {}
                        """.format(tuple(df.callid))
clint_voice_emotion = pd.read_sql_query(query_clint_voice_emotion, engine)
# Replace this with your SQL query
query_agent_voice_emotion = """
                    SELECT * FROM public.call_mood_agent
                    WHERE callid IN {}
                        """.format(tuple(df.callid))
agent_voice_emotion = pd.read_sql_query(query_agent_voice_emotion, engine)


In [7]:

   # Extract emotions and counts
voice_emotions_clinet = pd.DataFrame(clint_voice_emotion.emotion.value_counts()).reset_index()['emotion'].tolist()
voice_counts_client = pd.DataFrame(clint_voice_emotion.emotion.value_counts()).reset_index()['count'].tolist()
# Extract emotions and counts
voice_emotions_agent = pd.DataFrame(agent_voice_emotion.emotion.value_counts()).reset_index()['emotion'].tolist()
voice_counts_agent = pd.DataFrame(agent_voice_emotion.emotion.value_counts()).reset_index()['count'].tolist()
# Extract emotions and counts

In [8]:
df[df.alert == 0]

,callid,starttime,agentid,callscore,duration,endtime,id,topic,ironypercent,clientinterrupts,agentinterrupts,silencepercent,hatespeechpercent,clientspeechduration,agentspeechduration,customer_satisfaction_rate,agent_performance_rate,alert,satisfied
4,405,2024-01-04 03:00:45.783885,170226,82,8,2024-01-04 03:01:30.783885,5,YourTopic4,56,0,1,26,60,23,8,11,87,0,False
6,407,2023-02-09 13:42:45.783885,170224,81,130,2023-02-09 13:43:58.783885,7,YourTopic3,71,0,1,84,7,24,30,90,84,0,True
7,408,2023-09-12 21:08:45.783885,170223,50,117,2023-09-12 21:09:27.783885,8,YourTopic7,99,0,1,84,98,11,0,87,85,0,True
8,409,2023-10-17 09:16:45.783885,170225,47,161,2023-10-17 09:18:25.783885,9,YourTopic0,36,1,1,18,17,5,26,54,100,0,True
10,411,2023-12-23 22:38:45.783885,170227,19,88,2023-12-23 22:39:28.783885,11,YourTopic9,42,1,1,42,14,20,13,6,84,0,True
13,414,2023-08-04 06:48:45.783885,170228,50,174,2023-08-04 06:49:17.783885,14,YourTopic1,92,1,1,54,51,5,15,32,29,0,False
17,418,2023-02-27 12:42:45.783885,170226,18,62,2023-02-27 12:43:46.783885,18,YourTopic0,88,0,1,96,9,14,27,51,95,0,True
20,421,2023-12-30 04:47:45.783885,170223,36,122,2023-12-30 04:48:14.783885,21,YourTopic6,75,0,1,3,69,16,5,29,70,0,False
24,425,2023-07-28 09:57:45.783885,170228,45,67,2023-07-28 09:58:54.783885,25,YourTopic1,81,1,0,90,3,25,24,2,9,0,False
28,429,2023-11-04 09:11:45.783885,170222,14,5,2023-11-04 09:12:15.783885,29,YourTopic5,69,1,1,76,19,19,23,57,36,0,True


In [13]:
agent_text_emotion

,id,callid,speechtext,sentiment,irony,starttime,endtime,hatelevelpercent,meanemotion
0,1,401,Random text 1,neutral,0,2024-01-13 18:18:27.824429,2024-01-13 17:54:37.138510,57,happy
1,2,402,Random text 2,negative,1,2024-01-13 18:24:25.589352,2024-01-13 18:05:48.456728,26,sad
2,3,403,Random text 3,neutral,1,2024-01-13 18:01:53.055021,2024-01-13 18:20:46.633131,73,neutral
3,4,404,Random text 4,positive,0,2024-01-13 17:44:33.903565,2024-01-13 18:06:54.104108,71,sad
4,5,405,Random text 5,positive,0,2024-01-13 17:57:39.193944,2024-01-13 17:44:45.626066,19,neutral
...,...,...,...,...,...,...,...,...,...
95,96,496,Random text 96,negative,0,2024-01-13 18:08:05.094322,2024-01-13 18:10:10.022695,85,sad
96,97,497,Random text 97,positive,0,2024-01-13 17:55:21.196371,2024-01-13 17:34:38.393578,4,neutral
97,98,498,Random text 98,neutral,1,2024-01-13 17:58:17.081540,2024-01-13 17:36:39.449575,57,sad
98,99,499,Random text 99,neutral,0,2024-01-13 18:20:33.511802,2024-01-13 18:23:48.743449,58,sad


In [12]:

# Create data for Pie chart
labels = ['Hateful', 'Normal']
values = [60, 40]
# Create Pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='none')])
fig.update_layout(
    width=300,
    height=400,
    plot_bgcolor='rgb(16,18,22)',
    paper_bgcolor='rgb(16,18,22)'
)
fig.show()


In [16]:
# Create data for Pie chart
labels = ['Hateful', 'Normal']
values = [60, 40]  # Negative value for "Hateful"
colors = ['#A52A2A', '#006400']  # Red for negative, Green for positive

# Create Pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='none', marker=dict(colors=colors))])
fig.update_layout(
    width=300,
    height=400,
    plot_bgcolor='rgb(16,18,22)',
    paper_bgcolor='rgb(16,18,22)'
)
fig.show()

In [50]:
def get_ind_data(call_id):
    query_call_info = """SELECT * FROM public.callinfo
                         where callid = {}""".format(call_id)
    query_call = """SELECT * FROM public.call
                         where callid = {}""".format(call_id)

    df_call_info = pd.read_sql_query(query_call_info, engine)
    df_call = pd.read_sql_query(query_call, engine)

    df = pd.merge(df_call,df_call_info,how='left',on='callid')

    # Replace this with your SQL query
    query_clint_text_emotion = """
                        SELECT * FROM public.call_mood_agent
                        WHERE callid = {}
                            """.format(call_id)
    clint_text_emotion = pd.read_sql_query(query_clint_text_emotion, engine)
    # Replace this with your SQL query
    query_agent_text_emotion = """
                        SELECT * FROM public.call_mood_agent
                        WHERE callid = {}
                            """.format(call_id)
    agent_text_emotion = pd.read_sql_query(query_agent_text_emotion, engine)

    call_id
    topic = df.topic[0]
    duration = df.duration[0]
    datetime = df.starttime[0]
    customer_satisfaction_rate = df.customer_satisfaction_rate[0]
    agent_performance_rate = df.agent_performance_rate[0]
    callscore = df.callscore[0]
    emotion_client = list(clint_text_emotion.emotion_in_numbers)
    emotion_agent = list(agent_text_emotion.emotion_in_numbers)

    dict_met = {'call_id':call_id,'topic':topic,'duration':duration,'datetime':datetime,
                'customer_satisfaction_rate':customer_satisfaction_rate,'agent_performance_rate':agent_performance_rate,
                'callscore':callscore,'emotion_client':emotion_client,'emotion_agent':emotion_agent}
    return dict_met

In [51]:
get_ind_data(404)

{'call_id': 404,
 'topic': 'YourTopic0',
 'duration': 130,
 'datetime': Timestamp('2023-10-16 19:52:45.783885'),
 'customer_satisfaction_rate': 11,
 'agent_performance_rate': 22,
 'callscore': 53,
 'emotion_client': [4,
  4,
  4,
  2,
  4,
  4,
  2,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  1,
  1,
  2,
  1,
  4,
  1,
  4,
  2,
  4,
  4,
  1,
  4,
  3,
  4,
  4],
 'emotion_agent': [4,
  4,
  4,
  2,
  4,
  4,
  2,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  1,
  1,
  2,
  1,
  4,
  1,
  4,
  2,
  4,
  4,
  1,
  4,
  3,
  4,
  4]}

In [15]:
df

,callid,starttime,agentid,callscore,duration,endtime,id,topic,ironypercent,clientinterrupts,agentinterrupts,silencepercent,hatespeechpercent,clientspeechduration,agentspeechduration,customer_satisfaction_rate,agent_performance_rate,alert,satisfied
0,405,2024-01-04 03:00:45.783885,170226,82,8,2024-01-04 03:01:30.783885,5,YourTopic4,56,0,1,26,60,23,8,11,87,0,False


405